# QSwitch usage

In [1]:
import pprint
pp = pprint.PrettyPrinter()
import numpy as np
import qcodes
from qcodes_contrib_drivers.drivers.QDevil import QSwitch

Connect via USB:

In [1]:
qswitch_addr = '/dev/cu.usbmodem14101'
qswitch = QSwitch.QSwitch('switch', visalib='@py', address=f'ASRL{qswitch_addr}::INSTR')

Connected to: Quantum Machines QSwitch (serial:4, firmware:0.160) in 0.07s


Or connect via Ethernet:

In [2]:
qswitch_addr = '192.168.8.16'
qswitch = QSwitch.QSwitch('switch', visalib='@py', address=f'TCPIP::{qswitch_addr}::5025::SOCKET')

Connected to: Quantum Machines QSwitch (serial:5, firmware:0.160) in 0.13s


When the QSwitch is reset, all lines are grounded:

In [3]:
qswitch.reset()
qswitch.state()

'(@1!0:24!0)'

Or expressed as a Python array:

In [5]:
qswitch.closed_relays()

[(1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 0),
 (6, 0),
 (7, 0),
 (8, 0),
 (9, 0),
 (10, 0),
 (11, 0),
 (12, 0),
 (13, 0),
 (14, 0),
 (15, 0),
 (16, 0),
 (17, 0),
 (18, 0),
 (19, 0),
 (20, 0),
 (21, 0),
 (22, 0),
 (23, 0),
 (24, 0)]

Beep and blink on SCPI command errors:

In [6]:
qswitch.error_indicator('on')

## Manipulation by numbers

Connect and unground line 23:

In [7]:
qswitch.close_relay(23, 9)
qswitch.open_relay(23, 0)
qswitch.state()

'(@1!0:22!0,24!0,23!9)'

Tap off line 23 to BNC 2:

In [8]:
qswitch.close_relay(23, 2)
qswitch.state()

'(@1!0:22!0,24!0,23!9,23!2)'

Multiple relays at once:

In [9]:
qswitch.open_relays([(22, 0), (24, 0)])
qswitch.state()

'(@1!0:21!0,23!9,23!2)'

## Arrangements

At a higher level of abstraction, lines and break-out BNCs can be given names.

In [14]:
qswitch.arrange(
    breakouts={'DMM': 2, 'VNA': 7},  # BNC 2 connected to DMM & BNC 7 to VNA
    lines={'plunger': 23, 'sensor': 5})  # Give names to lines 23 & 5

There are specialised functions for manipulating relays by name:

In [ ]:
qswitch.connect('plunger')

which is equivalent to

In [13]:
qswitch.close_relay(23, 9)
qswitch.open_relay(23, 0)

that is, connect the device-under-test to input line 23, and then unground it.

For the BNC breakout:

In [15]:
qswitch.breakout('sensor', 'DMM')

which is equivalent to

In [13]:
qswitch.close_relay(5, 2)
qswitch.open_relay(5, 0)

that is, connect the device-under-test line 5 to BNC 2, and then unground it.

For convenience, the default name of a line or BNC is just its number, so eg.

In [12]:
qswitch.breakout('24', '1')

is equivalent to

In [16]:
qswitch.close_relay(24, 1)
qswitch.open_relay(24, 0)

Moreover, any breakouts are also disconnected when grounding:

In [ ]:
qswitch.ground('sensor')

will be equivalent to

In [ ]:
qswitch.close_relay(5, 0)
qswitch.open_relays([(5, 2), (5, 9)])

## Monitors

There is a pseudo parameter dedicated to monitoring:

In [ ]:
qswitch.overview()

In [ ]:
monitor = qcodes.Monitor(qswitch.overview)

## Autosave

When turning on autosave, the state will be restored over restart.

In [ ]:
qswitch.auto_save('on')
qswitch.auto_save('off')